Future of coding, part 1: convert relative imports to absolute.

In [12]:
import libcst as cst

project_name = 'opentaxforms'

In [13]:
# relative import from dot
# ImportFrom.module is None and ImportFrom.relative is nonempty.
cst.parse_module('from . import c')

Module(
    body=[
        SimpleStatementLine(
            body=[
                ImportFrom(
                    module=None,
                    names=[
                        ImportAlias(
                            name=Name(
                                value='c',
                                lpar=[],
                                rpar=[],
                            ),
                            asname=None,
                            comma=MaybeSentinel.DEFAULT,
                        ),
                    ],
                    relative=[
                        Dot(
                            whitespace_before=SimpleWhitespace(
                                value='',
                            ),
                            whitespace_after=SimpleWhitespace(
                                value='',
                            ),
                        ),
                    ],
                    lpar=None,
                    rpar=None,
                   

In [14]:
# relative import from named module
# ImportFrom.module is a Name and ImportFrom.relative is nonempty.
cst.parse_module('from .b import c')

Module(
    body=[
        SimpleStatementLine(
            body=[
                ImportFrom(
                    module=Name(
                        value='b',
                        lpar=[],
                        rpar=[],
                    ),
                    names=[
                        ImportAlias(
                            name=Name(
                                value='c',
                                lpar=[],
                                rpar=[],
                            ),
                            asname=None,
                            comma=MaybeSentinel.DEFAULT,
                        ),
                    ],
                    relative=[
                        Dot(
                            whitespace_before=SimpleWhitespace(
                                value='',
                            ),
                            whitespace_after=SimpleWhitespace(
                                value='',
                            ),
       

In [15]:
# absolute import
# ImportFrom.module is an Attribute and ImportFrom.relative is empty.
cst.parse_module('from a.b import c')

Module(
    body=[
        SimpleStatementLine(
            body=[
                ImportFrom(
                    module=Attribute(
                        value=Name(
                            value='a',
                            lpar=[],
                            rpar=[],
                        ),
                        attr=Name(
                            value='b',
                            lpar=[],
                            rpar=[],
                        ),
                        dot=Dot(
                            whitespace_before=SimpleWhitespace(
                                value='',
                            ),
                            whitespace_after=SimpleWhitespace(
                                value='',
                            ),
                        ),
                        lpar=[],
                        rpar=[],
                    ),
                    names=[
                        ImportAlias(
                            n

see the differences more easily via diff:
```
$ python -c "import libcst as cst; print(cst.parse_statement('from .b import x, y'))" > import_from_dot_b.cst
$ python -c "import libcst as cst; print(cst.parse_statement('from a.b import x, y'))" > import_from_a_dot_b.cst
$ vi -d import_from_dot_b.cst import_from_a_dot_b.cst
$ python -c "import libcst as cst; print(cst.parse_statement('from . import x, y'))" > import_from_dot.cst
$ python -c "import libcst as cst; print(cst.parse_statement('from a import x, y'))" > import_from_a.cst
$ vi -d import_from_dot.cst import_from_a.cst
```

In [16]:
class AbsolutifyImports(cst.CSTTransformer):
    def visit_ImportFrom(self, node):
        pass
    def leave_ImportFrom(self, original_node, updated_node):
        module = original_node.module
        if module is None:
            # None -> Name
            module = cst.Name(project_name)
        else:
            # Name -> Attribute
            module = cst.Attribute(
                value=cst.Name(project_name),
                attr=cst.Name(module.value),
            )
        return updated_node.with_changes(
            module=module,
            relative=[],
        )

transformer = AbsolutifyImports()

In [17]:
relative_code = cst.parse_module('from . import ut')
absolutified_code = relative_code.visit(transformer)
absolutified_code

Module(
    body=[
        SimpleStatementLine(
            body=[
                ImportFrom(
                    module=Name(
                        value='opentaxforms',
                        lpar=[],
                        rpar=[],
                    ),
                    names=[
                        ImportAlias(
                            name=Name(
                                value='ut',
                                lpar=[],
                                rpar=[],
                            ),
                            asname=None,
                            comma=MaybeSentinel.DEFAULT,
                        ),
                    ],
                    relative=[],
                    lpar=None,
                    rpar=None,
                    semicolon=MaybeSentinel.DEFAULT,
                    whitespace_after_from=SimpleWhitespace(
                        value=' ',
                    ),
                    whitespace_before_import=SimpleWhitespace(

In [18]:
absolutified_code.code
    # errors I ran into, and how to fix:
    # TypeError: 'NoneType' object is not iterable
        # relative=None  ->  relative=[]
    # AttributeError: 'str' object has no attribute '_codegen'
        # module.value -> cst.Name(module.value)

'from opentaxforms import ut'

In [19]:
# more compact round trip
cst.parse_module('from . import ut').visit(transformer).code

'from opentaxforms import ut'

In [20]:
cst.parse_module('from .ut import Bag').visit(transformer).code

'from opentaxforms.ut import Bag'

- benefits of unlocking the information in our code
    - refactor the code
        - eg split a service call into user-awaited vs other processing to reduce latency.
    - query the code
        - eg are all our biz rules really confined to one module as we designed?
    - test the code
        - generate new tests
    - optimize the design
        - given an input sequence, a loss function, and a set of allowed refactorings, minimize!
    - envisioning a future in which performance and memory optimizations [eg caching, sharding] are off-the-shelf refactorings.
- links
    - libcst
        - v 0.3
        - up to python 3.8 grammar
        - https://github.com/Instagram/LibCST/
        - https://instagram-engineering.com/static-analysis-at-scale-an-instagram-story-8f498ab71a0c
        - https://chairnerd.seatgeek.com/refactoring-python-with-libcst/
    - bowler is based on LibCST  https://pybowler.io
    - rope
        - https://github.com/python-rope/rope
    - RedBaron  https://github.com/PyCQA/redbaron
        - based on Baron
    - Baron
        - https://github.com/PyCQA/baron
        - Baron is a Full Syntax Tree (FST) library for Python. By opposition to an AST which drops some syntax information in the process of its creation (like empty lines, comments, formatting), a FST keeps everything and guarantees the operation fst_to_code(code_to_fst(source_code)) == source_code. 
        - Think of Baron as the "bytecode of python source code" and RedBaron as some sort of usable layer on top of it.
        - up to python 3.7 grammar
    - astroid
        - ast-based foundation for pylint; can analyze code but is not reversible.
    - Bicycle Repair Man
        - last released 2004
        - http://bicyclerepair.sourceforge.net/
    - uses ast to absolutify imports:  https://github.com/MarcoGorelli/absolufy-imports/
- want to talk about this series?  john@saponara.us